In [ ]:
import triton
import triton.language as tl

ModuleNotFoundError: No module named 'triton'

In [4]:
print(triton.__version__)

NameError: name 'triton' is not defined

In [ ]:
@triton.jit
def weighted_sum_fwd(
    x_ptr, weight_ptr,
    output_ptr,
    x_stride_row, x_stride_dim,
    weight_stride_dim,
    output_stride_row,
    ROWS,D,
    ROWS_TILE_SIZE:tl.constexpr, D_TILE_SIZE:tl.constexpr,#声明编译时常量，Tile分块的形状在编译时必须为已知
    ):
    row_tile_idx = tl.program_id(0)#检查正在运行哪个thread block，即获取当前线程块处理的张量子块

    x_block_ptr = tl.make_block_ptr(
        x_ptr,
        shape=(ROWS, D,),
        strides = (x_stride_row, x_stride_dim),
        offsets = (row_tile_idx * ROWS_TILE_SIZE, 0),
        block_shape = (ROWS_TILE_SIZE, D_TILE_SIZE),
        order=(1,0),#列优先的存储顺序
    )
    weight_block_ptr = tl.make_block_ptr(
        weight_ptr,
        shape=(D,),
        strides = (weight_stride_dim,),
        offsets=(0,),
        block_shape=(D_TILE_SIZE,),
        order=(0,),#行优先
    )
    output_block_ptr=tl.make_bolcj_ptr(
        output_ptr,
        shape=(ROWS,),
        strides = (output_stride_row,),
        offsets = (row_tile_idx*ROWS_TILE_SIZE,),
        block_shape=(ROWS_TILE_SIZE,),
        order=(0,),
    )

    #initialize a buffer to write to
    output = tl.zeros((ROWS_TILE_SIZE,), dtype=tl.float32)

    for i in range(tl.cdiv(D,D_TILE_SIZE)):#向上取整的除法
        #load the current block pointer
        #考虑行、列无法整除块，需要对2个维度进行边界检查
        row = tl.load(x_block_ptr, boundary_check=(0,1), padding_option = 'zero')#从指向的内存位置加载数据，只对第二个维度进行边界检查，如果超出边界，填充为0
        weight = tl.load(weight_block_ptr, boundary_check=(0,), padding_option='zero')

        #compute the weighted sum of the row
        output += tl.sum(row*weight[None,:], axis = 1)

        #移动指针到下一个块
        x_block_ptr = x_block_ptr.advance(0, D_TILE_SIZE)
        weight_block_ptr = weight_block_ptr.advance(ROWS_TILE_SIZE,)
    
    tl.store(output_block_ptr, output, boundary_check=(0,))


#wrap the kernel in a pytorch autograd function   
import torch, einops
from einops import rearrange
class WeightedSumFunc(torch.autofrad.Function): #torch.autofrad.Function是 PyTorch 中用于实现自定义前向和反向传播逻辑的基类。
    @staticmethod 
    #静态方法不依赖类实例，可以直接通过类名调用。
    def forward(ctx, x, weight):
        D, output_dims = x.shape[-1, ], x.shape[:-1]

        input_shape = x.shape
        #输入二维化
        x=rearrange(x, '... d -> (...) d') # '... d -> (...) d' 表示将所有前面的维度合并为一个维度，最后一维保持不变。

        ctx.save_for_backward(x, weight)

        assert len(weight.shape) ==1 and weight.shape[0]==D, 'Dimentsion mismatch'
        assert x.is_cuda and weight.is_cuda, 'Expected CUDA tensors'
        assert x.is_contiguous(), 'Our pointer arithmetic will assume contiguous x'

        ctx.D_TILE_SIZE = triton.next_power_of_2(D)//16 #将 D 向上取整到最近的 2 的幂，再将结果除以16，表示每个线程块处理的列数
        ctx.ROWS_TILE_SIZE=16 # 每个线程同时处理16行数据
        ctx.input_shape = input_shape
        
        #初始化一个空的结果张量，但元素不一定为0
        y = torch.empty(output_dims, decice = x.device)

        n_rows = y.numel()#输出y的元素总数

        #weight_sum_fwd函数是已经被@triton.jit装饰的函数体，因此在Triton中，可以使用[]，即用于指定内核的网格大小（grid size），即 GPU 上线程块的分布方式。
        #triton的内核函数调用语法如下：kernel[grid](args)
        #这里的grid是一个元组
        weighted_sum_fwd[(cdiv(n_rows, ctx.ROWS_TILE_SIZE),)](
            x, weight,
            y,
            x.stride(0), x.stride(1),#输出维度的步长
            weight.stride(0),
            y.stride(0),
            ROWS=n_rows, D=D,
            ROWS_TILE_SIZE = ctx.ROWS_TILE_SIZE, D_TILE_SIZE = ctx.D_TILE_SIZE, 
        )
        return y.view(input_shape[:-1])

@triton.jit
def weighted_sum_backward(
    x_ptr, weight_ptr, # input
    grad_output_ptr, # grad input
    grad_x_ptr, partial_grad_weight_ptr, # grad outputs
    stride_xr, stride_xd,
    stride_wd,
    stride_gr,
    stride_gxr, stride_gxd,
    stride_gwb, stride_gwd,
    NUM_ROWS, D,
    ROWS_TILE_SIZE:tl.constexpr, D_TILE_SIZE:tl.constexpr,
):
    row_title_idx = tl.program_id(0)
    n_row_tiles = tl.num_programs(0)

    #inputs
    grad_outputs_block_ptr = tl.make_block_ptr(
        grad_output_ptr,
        shape = (NUM_ROWS,), strides = (stride_gr,),
        offsets = (row_title_idx * ROWS_TILE_SIZE,),
        block_shape=(ROWS_TILE_SIZE,),
        order=(0,),
    )

    x_block_ptr = tl.make_block_ptr(
    x_ptr,
    shape=(NUM_ROWS, D,), strides=(stride_xr, stride_xd),
    offsets=(row_tile_idx * ROWS_TILE_SIZE, 0),
    block_shape=(ROWS_TILE_SIZE, D_TILE_SIZE),
    order=(1, 0),
    )

    weight_block_ptr = tl.make_block_ptr(
        weight_ptr,
        shape=(D,), strides=(stride_wd,),
        offsets=(0,),
        block_shape=(D_TILE_SIZE,),
        order=(0,),
    )

    grad_x_block_ptr = tl.make_block_ptr(
        grad_x_ptr,
        shape=(NUM_ROWS, D,), strides=(stride_gxr, stride_gxd),
        offsets=(row_tile_idx * ROWS_TILE_SIZE, 0),
        block_shape=(ROWS_TILE_SIZE, D_TILE_SIZE),
        order=(1, 0),
        )

    partial_grad_weight_block_ptr = tl.make_block_ptr(
        partial_grad_weight_ptr,
        shape=(n_row_tiles, D,), strides=(stride_gwb,stride_gwd),
        offsets=(row_tile_idx, 0),
        block_shape=(1, D_TILE_SIZE),
        order=(1,0),
    )

    for i in range(tl.cdiv(D,D_TILE_SIZE)):
        grad_output = tl.load(grad_outputs_block_ptr, boundary_check=(0,), padding_option='zero')#加载输出梯度

        #outer product for grad_x
        weight = tl.load(weight_block_ptr, boundary_check=(0,), padding_option='zero')
        grad_x_row = grad_output[:, None] * weight[None, :] #输出梯度与w的外积
        tl.store(grad_x_block_ptr, grad_x_row, boundary_check=(0,1))

        #权重梯度
        row = tl.load(x_block_ptr, boundary_check=(0,1), padding_option='zero')
        grad_weight_row = tl.sum(row*grad_output[:,None], axis=0, keep_dims=True)
        tl.store(partial_grad_weight_block_ptr, grad_weight_row, boundary_check=(1,))

        #移走下一个块的指针
        x_block_ptr = x_block_ptr.advance((0, D_TILE_SIZE))
        weight_block_ptr = weight_block_ptr.advance((D_TILE_SIZE,))
        partial_grad_weight_block_ptr = partial_grad_weight_block_ptr.advance((0,D_TILE_SIZE))
        grad_x_block_ptr = grad_x_block_ptr.advance((0, D_TILE_SIZE))


class WeightedSumFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weight):
        D, output_dims = x.shape[-1, ], x.shape[:-1]
        input_shape = x.shape
        #输入二维化
        x=rearrange(x, '... d -> (...) d') # '... d -> (...) d' 表示将所有前面的维度合并为一个维度，最后一维保持不变。

        ctx.save_for_backward(x, weight)

        assert len(weight.shape) ==1 and weight.shape[0]==D, 'Dimentsion mismatch'
        assert x.is_cuda and weight.is_cuda, 'Expected CUDA tensors'
        assert x.is_contiguous(), 'Our pointer arithmetic will assume contiguous x'

        ctx.D_TILE_SIZE = triton.next_power_of_2(D)//16 #将 D 向上取整到最近的 2 的幂，再将结果除以16，表示每个线程块处理的列数
        ctx.ROWS_TILE_SIZE=16 # 每个线程同时处理16行数据
        ctx.input_shape = input_shape
        
        #初始化一个空的结果张量，但元素不一定为0
        y = torch.empty(output_dims, device = x.device)

        n_rows = y.numel()#输出y的元素总数
        #weight_sum_fwd函数是已经被@triton.jit装饰的函数体，因此在Triton中，可以使用[]，即用于指定内核的网格大小（grid size），即 GPU 上线程块的分布方式。
        #triton的内核函数调用语法如下：kernel[grid](args)
        #这里的grid是一个元组

        weighted_sum_fwd[(cdiv(n_rows, ctx.ROWS_TILE_SIZE),)](
            x, weight,
            y,
            x.stride(0), x.stride(1),#输出维度的步长
            weight.stride(0),
            y.stride(0),
            ROWS=n_rows, D=D,
            ROWS_TILE_SIZE = ctx.ROWS_TILE_SIZE, D_TILE_SIZE = ctx.D_TILE_SIZE, 
        )
        return y.view(input_shape[:-1])

    @staticmethod
    def backward(ctx, grad_out):
        x, weight = ctx.saved_tensors
        ROWS_TILE_SIZE, D_TILE_SIZE,= ctx.ROWS_TILE_SIZE, ctx.D_TILE_SIZE,
        n_rows, D = x.shape #.shape返回元组，可以与解包的特性配合使用

        partial_grad_weight = torch.empty((cdiv(n_rows, ROWS_TILE_SIZE), D), device = x.device, dtype=x.dtype)
        grad_x = torch.empty_like(x)
        #torch.empty_like用于创建张量的核心函数，核心作用是生成一个和指定张量「形状、数据类型、设备、布局完全相同」但未初始化的空张量——“未初始化” 意味着张量内的值是内存中的随机垃圾值，不会自动填充 0 或其他默认值
        #torch.empty()是手动指定，而不是模仿已有的张量创建未初始化张量
        weighted_sum_backward[(cdiv(n_rows, ROWS_TILE_SIZE),)](
            x, weight,
            grad_out,
            grad_x, partial_grad_weight,
            x.stride(0), x.stride(1),
            weight.stride(0),
            grad_out.stride(0),
            grad_x.stride(0),grad_x.stride(1),
            partial_grad_weight.stride(0), partial_grad_weight.stride(1),
            NUM_ROWS = n_rows, D=D,
            ROWS_TILE_SIZE=ROWS_TILE_SIZE, D_TILE_SIZE=D_TILE_SIZE,
        )
        grad_weight = partial_grad_weight.sum(axis=0)
        return grad_x, grad_weight
    

if '__name__' == 'main':
    x = torch.randn(3,2,device ='cuda')
    weight = torch.randn(2,3,device ='cuda')
    f_weightedsum = WeightedSumFunc.apply # torch.autograd.Function 的一个方法，用于调用自定义的前向和反向传播逻辑。
    result = f_weightedsum(x, weight) #ctx因为是autograd.Function的入口封装方法打来的，因此会自动创建一个ctx（上下文对象），因此只需要关注ctx后面的参数
    #forward 和 backward 就是通过固定的函数名称（关键字） 被 PyTorch 识别的 ——PyTorch 内部会严格检查继承自 autograd.Function 的类是否实现了名为 forward 和 backward 的静态方法，
    # 这是 PyTorch 自动微分机制的 “约定式编程” 规则










        
